<a href="https://colab.research.google.com/github/pydevcasts/MLHub/blob/master/khhoseynpour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
یک روش جدید برای تشخیص نفوذ در شبکه‌های اینترنت اشیاء (IoT) با استفاده از *شبکه‌های عصبی کانولوشنی (CNN)* و *بهینه‌سازی ازدحام ذرات (APSO)* می‌پردازد. هدف این تحقیق شناسایی دقیق انواع مختلف حملات نفوذ است که توسط میزبان‌های کنترل‌شده توسط حمله‌کنندگان انجام می‌شود. روش APSO-CNN با بهینه‌سازی پارامترهای ساختاری CNN و معرفی یک معیار ارزیابی جدید، عملکرد بهتری نسبت به روش‌های سنتی مانند R-CNN، SVM و FNN نشان می‌دهد. نتایج نشان می‌دهد که APSO-CNN دقت شناسایی بالایی دارد و نرخ هشدار کاذب را کاهش می‌دهد، که این رویکرد را به ابزاری مؤثر و قابل اعتماد برای امنیت شبکه‌های IoT تبدیل می‌کند.
"""

In [2]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=d4969acfe641312b44ee11e3cf0b5f592c5fa9669d190677f9bbf47fb5d49ab9
  Stored in directory: /root/.cache/pip/wheels/bb/4f/ec/8970b83323e16aa95034da175454843947376614d6d5e9627f
Successfully built pyswarm


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyswarm import pso
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# تنظیمات اولیه
num_classes = 8  # تعداد کلاس‌های حملات شبکه
input_shape = (23, 5, 1)  # ویژگی‌های ورودی

# داده‌های آموزش و اعتبارسنجی (کاهش اندازه داده‌ها)
x_train = np.random.random((20, 23, 5, 1))  # داده‌های آموزشی با 20 نمونه
y_train = np.random.randint(0, num_classes, 20)  # برچسب‌های آموزشی
y_train = to_categorical(y_train, num_classes)  # تبدیل برچسب‌ها به قالب One-hot

x_val = np.random.random((5, 23, 5, 1))  # داده‌های اعتبارسنجی با 5 نمونه
y_val = np.random.randint(0, num_classes, 5)  # برچسب‌های اعتبارسنجی
y_val = to_categorical(y_val, num_classes)  # تبدیل برچسب‌ها به قالب One-hot

# تابع تعریف مدل CNN
def create_model(params):
    filters, kernel_size, dropout_rate, f2_neurons, f3_neurons, batch_size, learning_rate = params

    model = keras.Sequential([
        layers.Conv2D(filters=int(filters), kernel_size=(int(kernel_size), int(kernel_size)), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(int(f2_neurons / 4), activation='relu'),  # کاهش نورون‌ها
        layers.Dropout(float(dropout_rate)),
        layers.Dense(int(f3_neurons / 4), activation='tanh'),  # کاهش نورون‌ها
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.SGD(learning_rate=float(learning_rate)),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# تابع ارزیابی برای بهینه‌سازی APSO
def fitness_function(params):
    model = create_model(params)
    # استفاده از EarlyStopping برای کاهش زمان آموزش
    early_stopping = EarlyStopping(monitor='val_loss', patience=1, verbose=0)
    history = model.fit(x_train, y_train, batch_size=int(params[5]), epochs=5, validation_data=(x_val, y_val), verbose=0, callbacks=[early_stopping])
    return history.history['val_loss'][-1]  # مقدار تابع خطا

# تعیین محدوده مقادیر هایپرپارامترها
lb = [100, 1, 0.4, 64, 64, 4, 0.01]  # محدوده‌های پایین (کاهش نورون‌ها)
ub = [600, 5, 0.8, 256, 256, 16, 1]  # محدوده‌های بالا

# اجرای الگوریتم APSO
best_params, _ = pso(fitness_function, lb, ub)

# ایجاد مدل نهایی با بهترین هایپرپارامترها
best_model = create_model(best_params)

# آموزش مدل نهایی
history = best_model.fit(x_train, y_train, batch_size=int(best_params[5]), epochs=10, validation_data=(x_val, y_val), verbose=1)

# ارزیابی مدل نهایی
test_loss, test_acc = best_model.evaluate(x_val, y_val)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

# ذخیره مدل نهایی
best_model.save("final_model.h5")

# نمایش نتایج آموزش
# ترسیم دقت و خطا
plt.figure(figsize=(12, 4))

# دقت
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

# خطا
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
